## konlpy 설치

In [193]:
!pip install konlpy

## 라이브러리 import

In [194]:
import pandas as pd
import numpy as np
import pickle

from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 데이터 로드

In [195]:
# dataframe print 이쁘게
pd.set_option('display.unicode.east_asian_width', True)

In [196]:
df = pd.read_csv('/content/datasets/naver_news_titles_210616.csv', index_col=0)
print(df)
print(df.info())

                                                                           title  category
0        김총리  새 거리두기 방안  일상에 큰 변화 될 것                           Politics
1         백신 유급휴가비  지원한다 법개정안 복지위 통과                          Politics
2       쇄신   변화  강조한 송영길  개혁 속도조절  국민공감대 우선 ......         Politics
3        소통수석  열린자세로  의 중요 이웃국 역할할 것  종합 ......              Politics
4                 큐어백  백신 도입     한국  생산 거점                           Politics
...                                                  ...                               ...
28304  2만4000년간 죽지 않은  좀비 가 시베리아에서 나타났다......                       IT
28305   PLAY IT  갤럭시북 경험  키보드에서도 블루투스 키보드  트리오500  써보...        IT
28306   PLAY IT   카카오판 클럽하우스  음  체험기  작지만 큰차이 ......                 IT
28307   PLAY IT  홈쇼핑 고객 정착 가능할까  CJ온스타일  이용해보니......                IT
28308   PLAY IT  크롬 잡겠다는 네이버 웨일  신기능 사용해보니......                     IT

[28309 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28309 entries, 0 to 28308
Dat

## 전처리

## 중복값 제거

In [197]:
df.drop_duplicates(subset=['title'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df['title'].duplicated().sum())

0


### feature, target 분리

In [198]:
X = df['title']
Y = df['category']

### target encoding 및 encoder 저장

In [199]:
# label encoding
encoder = LabelEncoder()
labeled_Y = encoder.fit_transform(Y)
label = encoder.classes_
print(label)
print(labeled_Y)

['Culture' 'Economic' 'IT' 'Politics' 'Social' 'World']
[3 3 3 ... 2 2 2]


In [200]:
# 나중에 다른 문장을 토큰화할때 동일한 classes_로 인코딩해야함 -> 학습시킨 encoder 저장
# pickle로 저장 / 불러오기시 원본의 데이터 타입 그대로 유지
with open('/content/datasets/category_encoder.pickle', 'wb') as f:
    pickle.dump(encoder, f)

In [201]:
# one-hot encoding
onehot_Y = to_categorical(labeled_Y)
print(onehot_Y)

[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


## 형태소 분석

In [202]:
# 첫 뉴스 타이틀 형태소 분석
okt = Okt()
print(type(X))
okt_X = okt.morphs(X[0])
print(X[0])
print(okt_X)

<class 'pandas.core.series.Series'>
김총리  새 거리두기 방안  일상에 큰 변화 될 것 
['김', '총리', '새', '거리', '두기', '방안', '일상', '에', '큰', '변화', '될', '것']


In [203]:
# 데이터 형태소 분석
for i in range(len(X)):
    X[i] = okt.morphs(X[i])
print(X)

0        [김, 총리, 새, 거리, 두기, 방안, 일상, 에, 큰, 변화, 될, 것]...
1        [백신, 유급, 휴가, 비, 지원, 한, 다, 법, 개정안, 복지, 위, 통과]...
2        [쇄신, 변화, 강조, 한, 송영길, 개혁, 속도, 조절, 국민, 공감, 대, 우선...
3        [소통, 수석, 열린, 자세, 로, 의, 중요, 이웃, 국, 역할, 할, 것, 종합...
4                 [큐어, 백, 백신, 도입, 한국, 생산, 거점]
                               ...                        
27070    [2만, 4000년, 간, 죽지, 않은, 좀비, 가, 시베리아, 에서, 나타났다]...
27071    [PLAY, IT, 갤럭시, 북, 경험, 키, 보드, 에서도, 블루투스, 키, 보드...
27072    [PLAY, IT, 카카오, 판, 클럽, 하우스, 음, 체험, 기, 작, 지만, 큰...
27073    [PLAY, IT, 홈쇼핑, 고객, 정착, 가능할까, CJ, 온스타일, 이용, 해보...
27074    [PLAY, IT, 크롬, 잡겠다는, 네이버, 웨일, 신, 기능, 사용, 해보니]...
Name: title, Length: 27075, dtype: object


In [204]:
# stopword 로드
stopwords = pd.read_csv('/content/datasets/stopwords.csv', index_col=0)
print(stopwords)

    stopword
0         아
1         휴
2     아이구
3     아이쿠
4     아이고
..       ...
783   없다는
784   않을까
785     있지
786   있으며
787   그대로

[787 rows x 1 columns]


In [205]:
# 첫 뉴스 타이틀에서 stopword에 해당하는 문제 제외
words = []
for word in okt_X:
    if word not in list(stopwords['stopword']):
        words.append(word)
print(words)

['김', '총리', '새', '거리', '두기', '방안', '일상', '큰', '변화', '될']


In [206]:
# 함수로 만들어 기존 데이터에 apply해 stopword 제거
def delete_stopwords(lst):
    words = []
    for word in lst:
        if word not in list(stopwords['stopword']) and len(word) > 1:
            words.append(word)
    return ' '.join(words)

X.apply(delete_stopwords)

0                            총리 거리 두기 방안 일상 변화
1                     백신 유급 휴가 지원 개정안 복지 통과
2           쇄신 변화 강조 송영길 개혁 속도 조절 국민 공감
3                  소통 수석 열린 자세 중요 이웃 역할 종합
4                            큐어 백신 도입 한국 생산 거점
                               ...                        
27070          2만 4000년 죽지 않은 좀비 시베리아 나타났다
27071    PLAY IT 갤럭시 경험 보드 에서도 블루투스 보드 트리오 500 보니...
27072                 PLAY IT 카카오 클럽 하우스 체험 차이
27073    PLAY IT 홈쇼핑 고객 정착 가능할까 CJ 온스타일 이용 해보니...
27074    PLAY IT 크롬 잡겠다는 네이버 웨일 기능 사용 해보니...
Name: title, Length: 27075, dtype: object

In [207]:
# 또는 for문으로 stopword 제거
for i in range(len(X)):
    result = []
    for j in range(len(X[i])):
        if len(X[i][j]) > 1:
            if X[i][j] not in list(stopwords['stopword']):
                result.append(X[i][j])
    X[i] = ' '.join(result)
print(X)

0                            총리 거리 두기 방안 일상 변화
1                     백신 유급 휴가 지원 개정안 복지 통과
2           쇄신 변화 강조 송영길 개혁 속도 조절 국민 공감
3                  소통 수석 열린 자세 중요 이웃 역할 종합
4                            큐어 백신 도입 한국 생산 거점
                               ...                        
27070          2만 4000년 죽지 않은 좀비 시베리아 나타났다
27071    PLAY IT 갤럭시 경험 보드 에서도 블루투스 보드 트리오 500 보니...
27072                 PLAY IT 카카오 클럽 하우스 체험 차이
27073    PLAY IT 홈쇼핑 고객 정착 가능할까 CJ 온스타일 이용 해보니...
27074    PLAY IT 크롬 잡겠다는 네이버 웨일 기능 사용 해보니...
Name: title, Length: 27075, dtype: object


## 토큰화 및 저장

In [208]:
# stopword 제거한 뉴스 타이틀 토큰화
token = Tokenizer()
token.fit_on_texts(X)
tokened_X = token.texts_to_sequences(X)
print(tokened_X[0])

[281, 74, 138, 873, 603, 126]


In [209]:
# 나중에 다른 문장을 토큰화할때 같은 단어는 같은 숫자로 매핑해야함 -> 학습시킨 Tokenizer 저장
# pickle로 저장 / 불러오기시 원본의 데이터 타입 그대로 유지
with open('/content/datasets/news_token.pickle', 'wb') as f:
    pickle.dump(token, f)

In [210]:
# 단어의 개수 출력 / padding한 0을 포함하기 위해 +1
wordsize = len(token.word_index) + 1  # word_index는 토큰화한 단어와 숫자 보여줌 / 0을 포함하지 않고 1부터 시작
print(wordsize)

24151


## 문장 길이 맞춤

In [211]:
# 형태소가 가장 많은 문장의 단어 수
max = 0
for i in range(len(tokened_X)):
    if max < len(tokened_X[i]):
        max = len(tokened_X[i])
print(max)

27


In [212]:
# 길이가 맞게 앞쪽에 0 붙여줌
X_pad = pad_sequences(tokened_X, max)
print(X_pad[:10])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0   281    74   138
    873   603   126]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     2  4851  1001   105
   1953  2506   189]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0  2507   126  1030    66   636   219
   2825     7  1245]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0   923  1803  1804  3229  1421
   1547  1095     4]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0  1805     2   312
     11   156  2661]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0    66    19  1139 13780  3725  1139
   1059

## 데이터 저장

In [213]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_pad, onehot_Y, test_size = 0.1
)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(24367, 27)
(2708, 27)
(24367, 6)
(2708, 6)


In [214]:
xy = X_train, X_test, Y_train, Y_test
np.save(f'/content/datasets/news_data_max_{max}_size_{wordsize}.npy', xy)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
